# SED Fitting and Supernova Analysis Module 

**Lecturer:** Samaporn Tinyanont<br>
**Jupyter Notebook Authors:** Samaporn Tinyanont

This is a Jupyter notebook lesson taken from the NARIT HOWonOAR 2023.  


## Objective
Fit a black body function to multi-band photometry to measure the black body temperature, radius, and luminosity. Then reiterate this process to derive a bolometric light curve for a supernova.

## Key steps
- Plot spectral energy distribution (SED)
- Write a function that produce black body flux as a function of temperature and radius
- Use scipy curve fit to fit this function to provided data
- Reiterate this process for the entire light curve to make a bolometric light curve

## Required dependencies

See GROWTH school webpage for detailed instructions on how to install these modules and packages.  Nominally, you should be able to install the python modules with `pip install <module>`.  The external astromatic packages are easiest installed using package managers (e.g., `rpm`, `apt-get`).

### Python modules
* python 3
* astropy
* numpy
* matplotlib
* scipy

### External packages
None

In [1]:
import numpy as np

%matplotlib inline
import matplotlib.pyplot as plt
import os

import astropy.constants as const
import astropy.units as u
import astropy.io.ascii as asci

from scipy.optimize import curve_fit


## Load up the data and plot multi-band light curves

In [2]:
cwd = os.getcwd()
data_dir = os.path.join(cwd, 'data')

Load the provided multi-band light curve

First, we need to load our supernova light curve in many bands. In this case, the light curve is simplified and every epoch has data in every band. 

This file contains light curves in gri bands of SN 2015bn, obtained by the Las Cumbres Observatory. The data are taken from https://github.com/mnicholl/superbol/blob/master/example/2015bn_abs_ugriz.txt

In [6]:
# Load up our data and print it in the table format

sn_light_curve = asci.read(data_dir+'/2015bn_abs_ugriz.txt') 
sn_light_curve.show_in_notebook()

#TIP: You might want to rename columns in the txt files so they get read into the table correctly. 

idx,col1,col2,col3,col4,col5,col6,col7,col8,col9,col10,col11
0,-79.02,n,n,n,n,-19.50,0.28,n,n,n,n
1,-34.12,n,n,n,n,-21.42,0.18,n,n,n,n
2,-9.48,n,n,n,n,-21.70,0.04,n,n,n,n
3,-4.09,n,n,n,n,-21.74,0.02,n,n,n,n
4,0.4,n,n,n,n,-21.74,0.02,n,n,n,n
5,6.68,n,n,n,n,-21.77,0.04,n,n,n,n
6,19.25,n,n,n,n,-21.53,0.02,n,n,n,n
7,24.64,n,n,n,n,-21.36,0.02,n,n,n,n
8,30.03,n,n,n,n,-21.25,0.05,n,n,n,n
9,47.99,n,n,n,n,-21.12,0.12,n,n,n,n


In [ ]:
#You can select specific columns like so
sn_light_curve['epoch', 'g']

In [ ]:
#You can also dice the table to select desired row in many ways:
sn_light_curve[[0,1, 5, 6]]

In [ ]:
sn_light_curve[sn_light_curve['epoch'] < 0] #select all rows where the column epoch is < 0
# in general, in the bracket parenthesis can be any boolean array with the same length as the table
# and you will get the rows for which this boolean array value is True

In [ ]:
# Plot the light curve in different bands using different colors to mark different bands. 
# Take care to invert the y axis so that small magnitude (bright) is on top

plt.errorbar(sn_light_curve['epoch'], sn_light_curve['g'], yerr = sn_light_curve['gerr'], marker = 'o', color = 'g', label = 'g')
plt.errorbar(sn_light_curve['epoch'], ???, yerr = ???, marker = 'o', color = 'r', label = 'r')
plt.errorbar(sn_light_curve['epoch'], ???, yerr = ???, marker = 'o', color = 'c', label = 'i')

???ADD THE OTHER TWO BANDS???

plt.legend()

plt.gca().invert_yaxis()

plt.xlabel('Days from peak')
plt.ylabel('Absolute magnitude')

plt.title('SN 2015bn')

Here, you can see a typical light curve of a supernova: quick rise to a peak, then a long decline. This supernova is a rare subtype called a "superluminous" supernova, and this particular object is one of the best studied. 

These three light curves show the photometric evolution of this supernova in three different filters. In the next steps, we will estimate the bolometric light curve of this object. 

## Spectral Energy Distribution

A spectral energy distribution, or SED, is sort of a spectrum but with a very low resolution made with only a few filters. While it cannot show detailed spectral features like lines from different elements, it can be used to measure  the overall shape of the continuum, which is temperature dependent.   

Let's plot SED from three different epochs to see the temperature evolution of SN 2015bn. These three epochs selected represent early time, at peak, and late time SED of the supernova.  

In [ ]:
# First we define the three epochs we're interested in
epoch1 = sn_light_curve['epoch'] == -26.41 #this is just the first epoch
epoch2 = sn_light_curve['epoch'] == ??? #pick something close to peak
epoch3 = sn_light_curve['epoch'] == ??? #pick something else at later times

In [ ]:
# We now need to obtain the wavelength of these filters
# Again, SVO filter service is very useful: http://svo2.cab.inta-csic.es/theory/fps/index.php?mode=browse&gname=LasCumbres
# Here, we use lambda_mean in Angstrom

g_wave = 4749.16
r_wave = ???
i_wave = ???

??Add the u and z bands. 

In [ ]:
# Now we can plot the SED for each epoch

plt.errorbar([g_wave, r_wave, i_wave], \
             [sn_light_curve['g'][epoch1][0], sn_light_curve['r'][epoch1][0], sn_light_curve['i'][epoch1][0]], \
             yerr = [sn_light_curve['gerr'][epoch1][0], sn_light_curve['rerr'][epoch1][0], sn_light_curve['ierr'][epoch1][0]], \
            label = '-26.41 d')

plt.errorbar(???) #Do this for epoch 2

plt.errorbar(???) #Do this for epoch 3

plt.legend()
plt.xlabel('Wavelength ($\AA$)')
plt.ylabel('Absolute Magnitude')
plt.gca().invert_yaxis()

What do you see? Try plotting different epochs as well and see if you can notice the evolution of the SED shape. 

## Measuring temperature from the SED

We can fit an SED with the black body function to derive the temperature. This method assumes that the source emits like a black body, with no spectral features. In general, this works well for sources where spectral lines do not dominate the overall emission. For instance, supernovae near peak are very well explained by a black body. Old supernovae, on the ohter hands, have many strong emission lines; this makes the black body approximation less appropriate. For now, let's assume that a black body explains our SN 2015bn. 

In [ ]:
# First, we will write a helper function that takes an epoch of SN 2015bn light curve and gives us the SED. 
# Note that the code for the last plot looks very messy, and we don't want to have to do this every time. 

# We want a function that will take the desired epoch, say -26.41 day from the first epoch we used for the last plot, 
# and return two arrays: one with gri photometry and one with gri uncertainty.
# We should be able to recreate the last plot using two arrays from this function. 

# This function should also check if the input epoch has any data in it, and returns None if not.

def get_SED_from_epoch(desired_epoch):
    """
    This function takes the epoch of observation and returns g, r, i photometry and uncertainties in 
    appropriate arrays.
    """
    epoch = sn_light_curve['epoch'] == desired_epoch
    if np.sum(epoch) == 1:
        SED_at_epoch = np.array([???]) #Hint: just follow what was used in making the plot above.
        #You have to deal with the uneven data here
        Err_at_epoch = np.array([???])
        return SED_at_epoch, Err_at_epoch
    else:
        print("This epoch has no data.")
        return None

In [ ]:
# We will also define an array with wavelength so we don't have to rewrite that every time. 
wl_array = np.array([g_wave, r_wave, i_wave])

In [ ]:
# Now let's use this to recreate the SED from -26.41 day

SED_at_epoch, Err_at_epoch = get_SED_from_epoch(-26.41)

plt.errorbar(wl_array, SED_at_epoch, yerr = Err_at_epoch)
    
plt.xlabel('Wavelength ($\AA$)')
plt.ylabel('Absolute Magnitude')
plt.gca().invert_yaxis()

## Convert absolute magnitude to specific flux and wavelength to frequency

Recall that the AB magnitude scale means that an object with mag = 0 has a specific flux of $F_{\nu, 0} = 3631$ Jy. For an apparent magnitude, you can compute the observed flux from $F_\nu = F_{\nu, 0} 10^{-{\rm mag}/2.5}$. If the given magnitude is an absolute magnitude, this is an observed flux for a hypothetical observer located 10 parsec from the source. 

In [ ]:
#Quick astropy constant and unit tutorial: Let's compute E = m c^2 for 1 gram of mass

E = 1*u.g * const.c**2
print(E)

#You can do unit conversion like this:
print(E.to(u.J))

#You can also convert this back to a normal number
print(E.to(u.J).value)

In [ ]:
#Write a function that takes the absolute AB magnitude and converts to flux unit
def mag2flux(mag, magerr):
    flux = ??? #write a python version of the equation above. Use u.Jy to denote the unit. 
    fluxerr = np.abs(flux * np.log(10)/2.5 * magerr) #Extra credit: derive this!
    return flux, fluxerr

In [ ]:
#test out this function
mag2flux(SED_at_epoch, Err_at_epoch)

Flux_at_epoch, FluxErr_at_epoch = mag2flux(SED_at_epoch, Err_at_epoch)
print(Flux_at_epoch, FluxErr_at_epoch)

In [ ]:
#Now convert the wavelength array to frequency. We will get to use this soon. 
nu_array = ??? #use the astropy constant and unit package here as well

# You can convert unit like this:
nu_array.to(u.Hz)

## Black body radiation

The topic of black body radiation takes a good amount of time to explain. After all, this is one of the key issues that started quantum mechanics. In a nutshell, black body radiation is the spectrum you get from a perfect body that can absorb all light and reemit the energy via thermal radiation. The specific intensity of this spectrum is explained by the Planck function:

$B_\nu = \dfrac{2 h \nu^3}{c^2} \dfrac{1}{ e^{h \nu / k_B T} - 1} $

In [ ]:
#Uncomment the line below to see all the constants available:
#const?

In [ ]:
# Write a function that takes frequency nu and temperature T and outputs B_nu

def Bnu(nu, T):
    """
    Here, use astropy units and constants to your advantage so that unit conversion is taken care of
    """
    return ??? #Planck function goes here. Use the const? above to see the names for all the constants. 

In [ ]:
# Let's try out your function for 10,000 K
Bnu(nu_array, 10000*u.K)

We can now convert the specific intensity to flux. We discussed this briefly in the previous lecture on photometry, but in brief, for an isotropically emitting body, the flux at the surface of the object with radius $R$ is $F_{\nu, \rm surface} = \pi I_\nu$. In this case for black body radiation, $I_\nu$ is just the Planck function, so $F_{\nu, \rm surface} = \pi B_\nu $

At any distance $d > R$, the flux decreases according to the inverse square law: $F_\nu(d) = F_{\nu, \rm surface} \frac{R^2}{d^2} $

In [ ]:
# Write a function that takes frequency nu, temperature T, and black body radius R and gives the flux at 
# a distance d, default to 10 parsec
def Fnu(nu, T, R, d = 10*u.pc):
    return ??? #This should be F_nu(d) from the equation above. You should call the Bnu function above.

In [ ]:
#Test this function for R = 1e5 Rsun. Convert the unit to Jy.
#use a finer wavelength array

wlwl = np.linspace(3000,9000,10000)
nunu = const.c/(wlwl*u.angstrom)

Fnu(nunu, 10000*u.K, 1e5*u.R_sun).to(u.Jy)

In [ ]:
#Now make a plot comparing this black body curve to the flux of SN 2015bn
#Use wavelength in the x axis. Optical/IR astronomers dislike frequency!

#Here, you should tweak the temperature and radius to get the number similar to the observation.
#This does not have to be perfect, just close. 

T_guess = 10000*u.K
R_guess = 1e5*u.R_sun

plt.plot(wlwl ,Fnu(nunu, T_guess, R_guess).to(u.Jy), '-')
plt.errorbar(wl_array, Flux_at_epoch, FluxErr_at_epoch, marker = 'o')

plt.xlabel('Wavelength ($\AA$)')
plt.ylabel(r'$F_\nu$ (Jy)')

## Finally, actually fit the black body function to our data

We are almost at the end (of part 1)! Here, we will use a function fitting routine from scipy called curve_fit. Documentation for this routine can be found here: https://docs.scipy.org/doc/scipy/reference/generated/scipy.optimize.curve_fit.html

In short, it takes a function f that has a form f(x, a, b, c,...) where x is the independent variable (wavelenght or frequency here) and a, b, c, ... are parameters we are trying to fit. It also takes the independent and depedent variables x and y (here wavelength/frequency and flux), then it tries to adjust a, b, c, ... to make f(x, a, b, c, ...) as close as y as possible. 

In [ ]:
#We have to define the function to fit. This should be very similar to Fnu above, but for convenience, let it take
#wavelenght as x, and have T and R as the adjustable parameters

#Another quirk is that curve_fit cannot deal with astropy units. So let's give it just numerical value and we add 
#the units inside the function. Similarly, we also have to return a unitless number in Jansky

def Fnu_to_fit(wl, T, R):
    nu = const.c/(wl*u.angstrom)
    T = T*u.K
    R = R*u.R_sun
    
    flux = Fnu(nu, T, R)
    return flux.to(u.Jy).value

#Clearly define x and y and y_err for curve fit
x = ???
y = ???
y_err = ???

#Call curve_fit. Note that we use T_guess and R_guess as the initial value for the function.

result, covariance = curve_fit(Fnu_to_fit, x, y, sigma = y_err, absolute_sigma = True, p0 = (T_guess, R_guess))

In [ ]:
#Show the result
print(result)
print(covariance)

Here, the list result contains the best-fit value for our two parameters, T and R in the units of Kelvin and Rsun. The covariance matrix gives us the estimates of the variance of the two parameters. 

In [ ]:
best_T = result[0] * u.K
best_R = result[1] * u.R_sun
T_err = np.sqrt(np.diag(covariance))[0] *u.K
R_err = np.sqrt(np.diag(covariance))[1] *u.R_sun

In [ ]:
print("The best fit temperature is %d +- %d K"%(best_T.value, T_err.value))
print("The best fit radius is %e +- %e Rsun"%(best_R.value, R_err.value))

In [ ]:
#Remake the last plot, but also with the best fit value

plt.plot(wlwl ,Fnu(nunu, T_guess, R_guess).to(u.Jy), ':', label = 'First Guess')
plt.plot(wlwl ,Fnu(nunu, best_T, best_R).to(u.Jy), '-', label = 'Best Fit')

plt.errorbar(wl_array, Flux_at_epoch, FluxErr_at_epoch, marker = 'o', label = 'Data')

plt.legend()

plt.xlabel('Wavelength ($\AA$)')
plt.ylabel(r'$F_\nu$ (Jy)')

From the temperature and radius, we can calculate the black body luminosity via $L = 4 \pi R^2 \sigma T^4$ where $\sigma$ is the Stefan-Boltzmann constant. You get this by integrating $\pi B_\nu$ from $\nu = 0$ to $\infty$. See https://en.wikipedia.org/wiki/Stefan%E2%80%93Boltzmann_law

In [ ]:
#Write a function to compute black body luminosity and uncertainty given T and R

def blackbodyL(best_T,best_R,T_err, R_err):
    L_bb = ??? #again, use the function above
    L_bb_err = np.sqrt( (8*np.pi*best_R*const.sigma_sb*best_T**4 * R_err)**2 + \
                       (16*np.pi*best_R**2*const.sigma_sb*best_T**3 * T_err)**2) 
    return L_bb, L_bb_err

L_bb, L_bb_err =  blackbodyL(best_T,best_R,T_err, R_err)

In [ ]:
print(L_bb.to(u.erg/u.s))
print(L_bb_err.to(u.erg/u.s))

## Bolometric Light Curve

You have done this for one epoch. Write a code that can loop through all the epochs of SN 2015bn and produce three plots showing the evolution of black body luminosity, temperature, and radius. This is one of the most basic task one has to do for a supernova study. These three plots allow you to see the evolution of the cataclysmic fireball from a dead star, and can be compared with theoretical models to learn about its nature. 

When you do this, think about the real observations. Usually, you don't have all the same filters observation every night. Maybe you have g and r for one night, and only r and i for the next. How do you deal with that? Some supernovae are hidden behind a dust cloud (recall our discussion on dust extinction). How do you deal with that? There are many more tehcniques astronomers use to deal with our often incomplete and sometimes downright terrible datasets. Stars and galaxies are far away and we can do nothing but look. 


In [ ]:
#There are many ways to do this. My favorite is to write a for loop going through different epochs we have,
#fit for T and R, compute L, then save them into lists. 

In [ ]:
R = []
T = []
L = []
dR = []
dT = []
dL = []

for epoch in sn_light_curve['epoch']:
    #Get the SED at this epoch
    SED_at_epoch, Err_at_epoch = get_SED_from_epoch(epoch)
    #Convert to flux
    Flux_at_epoch, FluxErr_at_epoch = mag2flux(SED_at_epoch, Err_at_epoch)
    
    ##############
    #Fill in the gut here. Compute T and R from curve_fit, then L
    ##############
    
    R += [best_R.to(u.cm).value]
    T += [best_T.to(u.K).value]
    L += [best_L.to(u.erg/u.s).value]
    
    dR += [R_err.to(u.cm).value]
    dT += [T_err.to(u.K).value]
    dL += [L_err.to(u.erg/u.s).value]
    

In [ ]:
#Now we make the money plot, T L R as function of epoch
plt.errorbar(sn_light_curve['epoch'], T, dT)
plt.xlabel('Days from peak')
plt.ylabel('Temperature (K)')

In [ ]:
plt.errorbar(sn_light_curve['epoch'], L, dL)
plt.xlabel('Days from peak')
plt.ylabel('Luminosity (erg/s)')

plt.yscale('log')

In [ ]:
plt.errorbar(sn_light_curve['epoch'], R, dR)
plt.xlabel('Days from peak')
plt.ylabel('Blackbody Radius (cm)')

## Compare this to the published result for this supernova in Figure 17: https://arxiv.org/abs/1603.04748